# Ising model simulation with Monte Carlo method

In [ ]:
import os
import shutil
import numpy as np
import multiprocessing
from functools import partial

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
fdir = './out/ising/'
ffmt = 'pdf'
fdpi = 120

In [ ]:
def get_random_spins(size=None):
    '''
    Returns an array filled with +1 and -1 values, randomly sampled from
    a uniform distribution.
    '''
    return np.sign(np.random.random(size) - 0.5)

In [ ]:
def e_i(s_i, s_j, J=1):
    '''
    Calculates the interaction energy between two spins.
    '''
    return -J * s_i * s_j

In [ ]:
def calc_E(S, J=1, h=0):
    '''
    Calculates the total energy for a given spin configuration.
    '''
    # Size of the lattice
    Nr, Nc = S.shape[0], S.shape[1]
    # Relative indexes of neighbouring spins
    nb = np.array((-1, -Nc, 1, Nc))
    # Flattened indexes
    k = np.arange(0, S.size)
    k_nb = (np.repeat(k[:, np.newaxis], 4, axis=1) - nb) % (Nr*Nc)

    # Calculate E = -J * Sum_{<i,j>} s_i s_j - h * Sum_{i} s_i
    Sij = np.sum(np.multiply(S.reshape(-1, 1), S.flat[k_nb]), axis=1)
    E = -J*np.sum(Sij) / 4 - np.sum(h*S.flat[k])

    return E

In [ ]:
def calc_m(S):
    '''
    Calculates the total magnetization for a given spin configuration.
    '''
    return np.sum(S)

In [ ]:
def delta_E(S, i, j, J=1, h=0):
    '''
    Calculate the energy change after the flip of the (i, j) spin.
    '''
    # Size of the lattice
    Nr, Nc = S.shape[0], S.shape[1]
    # Relative indexes of neighbouring spins
    nb = np.array((-1, -Nc, 1, Nc))

    # Flattened indexes
    k = i*Nc + j
    k_nb = (k - nb) % (Nr*Nc)

    # Calculate the energy change
    dE = 2*J*np.sum(S.flat[k] * S.flat[k_nb]) + 2*h*S.flat[k]

    return dE

In [ ]:
def mcmc_step(S, beta=0.01, J=1, h=0):

    # Choose random indexes
    i = np.random.randint(0, S.shape[0])
    j = np.random.randint(0, S.shape[1])

    # Calculate energy difference on flip of spin_ij
    dE = delta_E(S, i, j, J, h)

    # If dE < 0, then execute the flip
    if(dE < 0):
        S[i, j] *= -1
    # If dE == 0, then randomly flip the spin
    elif(dE == 0):
        S[i, j] *= get_random_spins()
    # If dE > 0, choose a random number R, between 0 and 1
    # If R < e^(-beta * dE), then execute the spin flip
    # If not, then leave the spins' state untouched and continue with the next step
    elif(np.random.random() < np.exp(-beta * dE)):
        S[i, j] *= -1

    return S

In [ ]:
def mcmc(S, T, J, h, N_iter=1_000, save=False):

    # Calculate the dimensionless thermodynamic beta
    beta = 1 / T

    S_n = S.copy()
    for _ in range(N_iter):
        S_n = mcmc_step(S_n, beta=beta, J=J, h=h)

    if save:
        os.makedirs(fdir, exist_ok=True)
        np.save(file=os.path.join(fdir, f'spins_T{T}'), arr=S_n)

    return S_n

In [ ]:
Ts = np.array([0.1, 1.5, 2.5, 5])  # Temperature values
Nr, Nc = 400, 400  # Size of lattice
# The current positions of the spins
S = get_random_spins((Nr, Nc))
J = 1  # Coupling parameter (dimensionless)
h = 0  # External magnetic field
N_iter = 10_000_000  # Number of iterations

In [ ]:
%%time
partial_mcmc = partial(mcmc, S, J=J, h=h, N_iter=N_iter, save=True)
with multiprocessing.Pool(processes=4) as pool:
    Ss = pool.map(partial_mcmc, Ts)

In [ ]:
nrows, ncols = 1, Ts.size
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*7, nrows*7),
                         facecolor='ghostwhite')
fig.subplots_adjust(wspace=0.0)

spin_files = {
  f.split('T')[1].split('.npy')[0] : f for f in os.listdir(fdir) if '.npy' in f
}
for T, ax in zip(Ts, axes.flat):
    ax.axis('off')

    spins_n = np.load(file=os.path.join(fdir, spin_files[f'{T}']))
    ax.imshow(spins_n, cmap='Greys', interpolation='none')
    ax.set_title(f'{T = } K',
                 fontsize=16, fontweight='bold')
plt.show()

## Temperature dependence of energy and magnetization

In [ ]:
def mcmc_ms(S, T, J, h, N_iter=1_000):

    # Calculate the dimensionless thermodynamic beta
    beta = 1 / T

    # Storage for the measured physical values
    E = m = 0

    S_n = S.copy()
    for _ in range(N_iter):
        S_n = mcmc_step(S_n, beta=beta, J=J, h=h)
        E += calc_E(S_n, J=J, h=h)
        m += calc_m(S_n)

    return (E/N_iter, m/N_iter)

In [ ]:
%%time
# Temperature values
Ts = np.linspace(0.01, 10, 100)
# Size of the spin configuration
Nr, Nc = 50, 50
# The current positions of the spins
S = get_random_spins((Nr, Nc))

J = 1  # Coupling parameter (dimensionless)
h = 0  # External magnetic field

In [ ]:
# Number of iterations to find the equilibrium
N_iter_eq = 100_000

partial_mcmc_eq = partial(mcmc, S, J=J, h=h, N_iter=N_iter_eq, save=True)
with multiprocessing.Pool(processes=8) as pool:
    _ = pool.map(partial_mcmc_eq, Ts)
#Ss = np.array(Ss)

In [ ]:
# Numnber of iteration to measure the physical parameters
N_iter_ms = 50_000

partial_mcmc_ms = partial(mcmc_ms, S, J=J, h=h, N_iter=N_iter_ms)
with multiprocessing.Pool(processes=8) as pool:
    M = pool.map(partial_mcmc_ms, Ts)
M = np.array(M)

In [ ]:
E = M[:, 0]/(Nr*Nc)
m = M[:, 1]/(Nr*Nc)

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*7, nr*5), dpi=120)
axes = axes.flat
for ax in axes:
    ax.grid(True, ls='--', alpha=0.6)

axes[0].scatter(Ts, E, s=36)
axes[0].set_xlabel('Temperature', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Energy', fontsize=16, fontweight='bold')

axes[1].scatter(Ts, m, s=36)
axes[1].set_xlabel('Temperature', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Magnetization', fontsize=16, fontweight='bold')

plt.show()